In [1]:
from selenium import webdriver
import pandas as pd
import html5lib
import mplfinance as mpf
import matplotlib.pyplot as plt
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def Set_up_WebScraping(url):
    
    driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\week2\chromedriver.exe")
    driver.get(url)
    
    return driver

def save_csv(df,name):

    name += ".csv"
    df.to_csv(name,index=False)
    
    return "succes"

def save_excel(df,name):

    name += ".xlsx"
    df.to_excel(name,index=False)
    
    return "succes"

In [3]:
driver = Set_up_WebScraping("https://www.set.or.th/en/market/index/set100/overview")

C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_26380\2693298640.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\week2\chromedriver.exe")


# Statistics 

In [13]:
data = driver.get("https://www.set.or.th/en/market/product/stock/quote/PCC/financial-statement/company-highlights")

In [14]:
data = driver.page_source

In [15]:
data_df = pd.read_html(data)[1]

In [16]:
data_df

,Statistics as of,30 Dec 2022,10 Jan 2023
0,Last Price (Baht),3.32,3.36
1,Market Cap. (M.Baht),4072.38,4121.44
2,F/S Period (As of date),30 Sep 2022,30 Sep 2022
3,P/E,13.84,14.01
4,P/BV,1.35,1.36
5,Book Value per share (Baht),2.47,2.47
6,Dividend Yield (%),-,-


In [8]:
d = {}
m = [data_df['Statistics as of'].tolist(),data_df['30 Dec 2019'].tolist(),data_df['30 Dec 2020'].tolist(),data_df['30 Dec 2021'].tolist(),data_df['30 Dec 2022'].tolist(),data_df['06 Jan 2023'].tolist()]
k = 0
for i in data_df.columns:
    
    d[i] = m[k]
    k+=1
    
print(d)

KeyError: '06 Jan 2023'

In [21]:
d_c = pd.DataFrame(d)

In [9]:
data_new = (data_df.set_index("Statistics as of")).T

In [10]:
data_new

Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%)
30 Dec 2019,62.25,279378.00,30 Sep 2019,26.36,4.08,15.25,1.77
30 Dec 2020,47.75,214302.00,30 Sep 2020,19.04,3.22,14.84,1.68
30 Dec 2021,56.50,253572.00,30 Sep 2021,34.37,3.54,15.97,1.24
30 Dec 2022,71.00,318648.00,30 Sep 2022,32.49,4.02,17.66,0.85
09 Jan 2023,71.00,318648.00,30 Sep 2022,32.49,4.02,17.66,0.85


In [11]:
data_new["Symbol"] = "DTAC"

In [12]:
data_new = data_new.reset_index()

In [15]:
data_new = data_new.rename(columns = {"index":"Statistics as of"})

In [16]:
data_new

Statistics as of,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
0,30 Dec 2019,53.25,126085.94,30 Sep 2019,-,5.17,10.30,1.90,DTAC
1,30 Dec 2020,33.25,78729.72,30 Sep 2020,14.78,3.27,10.16,8.63,DTAC
2,30 Dec 2021,45.75,108327.35,30 Sep 2021,31.26,5.42,8.44,6.54,DTAC
3,30 Dec 2022,46.75,110695.16,30 Sep 2022,46.33,6.19,7.55,4.49,DTAC
4,05 Jan 2023,46.00,108919.31,30 Sep 2022,45.59,6.09,7.55,4.57,DTAC


In [19]:
data_new['Statistics as of'] = pd.to_datetime(data_new['Statistics as of'])

In [20]:
save_csv(data_new,"Statistics")

'succes'

# Function

In [13]:
def get_set50_100_df(driver):
    
    data = driver.page_source
    data_df = pd.read_html(data)[2]
    clean_column = []
    for column in data_df.columns:

        clean_column.append(column.replace("  (Click to sort Ascending)",""))
        
    data_df.columns = clean_column
    data_df = data_df.set_index('Symbol')
    return data_df

def operation_data(stock,dataframe):
    
    Statistics_df = (dataframe.set_index("Statistics as of")).T

    Statistics_df['Symbol'] = stock
    Statistics_df = Statistics_df.reset_index()
    Statistics_df = Statistics_df.rename(columns = {"index":"Statistics as of"})

    return Statistics_df

def get_Statistics_data(driver,stock,all_stock_year,all_stock_month):
    
    stock = stock.split()[0]
    url_target_financial = "https://www.set.or.th/en/market/product/stock/quote/" + stock + "/financial-statement/company-highlights"
    driver.get(url_target_financial)
    driver.execute_script("scrollBy(0,+5000);")
    time.sleep(3)
        
    Statistics_df = driver.page_source
    Statistics_df = pd.read_html(Statistics_df)[1]
    Statistics_df = operation_data(stock,Statistics_df)
    
    if '30' in (Statistics_df.iloc[[-1]]["Statistics as of"].tolist())[0][:2] :
        
        Statistics_df['Statistics as of'] = pd.to_datetime(Statistics_df['Statistics as of'])
        all_stock_year.append(Statistics_df)
    
    else : 

        Statistics_df['Statistics as of'] = pd.to_datetime(Statistics_df['Statistics as of'])
        all_stock_year.append(Statistics_df[:4])
        all_stock_month.append(Statistics_df.iloc[[-1]])
        
    print(stock)
    return driver , all_stock_year , all_stock_month

In [6]:
driver = Set_up_WebScraping("https://www.set.or.th/en/market/index/set100/overview")

C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_19504\2693298640.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\week2\chromedriver.exe")


In [7]:
data_df = get_set50_100_df(driver)

In [14]:
all_Statistics_year = []
all_Statistics_month = []

for stock in data_df.index[:10] :

    
    driver,all_Statistics_year,all_Statistics_month = get_Statistics_data(driver,stock,all_Statistics_year,all_Statistics_month)
    

table_year = pd.concat(all_Statistics_year)
table_month = pd.concat(all_Statistics_month)

AAV
ACE
ADVANC
AMATA
AOT
AP
AWC
BAM
BANPU
BBL


In [15]:
table_year

Statistics as of,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
0,2019-12-30,2.24,10864.00,30 Sep 2019,-,0.54,4.12,14.96,AAV
1,2020-12-30,2.36,11446.00,30 Sep 2020,-,0.76,3.11,-,AAV
2,2021-12-30,2.54,25091.57,30 Sep 2021,-,3.11,1.71,-,AAV
3,2022-12-30,3.06,37222.71,30 Sep 2022,-,6.81,0.45,-,AAV
0,2019-12-30,4.22,42942.72,-,62.46,-,-,-,ACE
1,2020-12-30,3.64,37040.64,30 Sep 2020,24.71,3.15,1.15,-,ACE
2,2021-12-30,3.50,35616.00,30 Sep 2021,25.58,2.75,1.27,1.37,ACE
3,2022-12-30,2.70,27475.20,30 Sep 2022,18.75,1.92,1.40,0.75,ACE
0,2019-12-30,213.00,633287.05,30 Sep 2019,20.45,10.22,20.84,3.32,ADVANC
1,2020-12-30,176.00,523345.56,30 Sep 2020,19.15,7.64,23.04,4.17,ADVANC


In [16]:
table_month

Statistics as of,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
4,2023-01-20,2.96,36006.29,30 Sep 2022,-,6.59,0.45,-,AAV
4,2023-01-20,2.66,27068.16,30 Sep 2022,18.41,1.89,1.41,0.76,ACE
4,2023-01-20,200.00,594841.95,30 Sep 2022,23.32,7.62,26.25,3.84,ADVANC
4,2023-01-20,20.10,23115.00,30 Sep 2022,10.07,1.21,16.68,1.99,AMATA
4,2023-01-20,74.75,1067856.08,30 Sep 2022,-,10.54,7.09,-,AOT
4,2023-01-20,11.40,35863.25,30 Sep 2022,6.27,1.01,11.31,4.39,AP
4,2023-01-20,6.10,195204.70,30 Sep 2022,57.16,2.40,2.54,0.18,AWC
4,2023-01-20,16.20,52359.09,30 Sep 2022,18.40,1.22,13.25,3.40,BAM
4,2023-01-20,12.70,107367.85,30 Sep 2022,2.42,0.96,16.46,2.52,BANPU
4,2023-01-20,151.50,289189.70,30 Sep 2022,10.31,0.57,267.22,2.31,BBL


In [103]:
save_csv(table_pice,"Statistics")

'succes'

In [104]:
table = pd.read_csv('Statistics.csv')

In [106]:
for i in range(len(table)):
    
    if table['F/S Period (As of date)'][i] == '-':
        
        continue
    
    table['F/S Period (As of date)'][i] = pd.to_datetime(table['F/S Period (As of date)'][i])

C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_16452\1973386235.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['F/S Period (As of date)'][i] = pd.to_datetime(table['F/S Period (As of date)'][i])


In [107]:
table

,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
0,2019-12-30,2.24,10864.00,2019-09-30 00:00:00,-,0.54,4.12,14.96,AAV
1,2020-12-30,2.36,11446.00,2020-09-30 00:00:00,-,0.76,3.11,-,AAV
2,2021-12-30,2.54,25091.57,2021-09-30 00:00:00,-,3.11,1.71,-,AAV
3,2022-12-30,3.06,37222.71,2022-09-30 00:00:00,-,6.81,0.45,-,AAV
4,2023-01-05,3.08,37466.00,2022-09-30 00:00:00,-,6.86,0.45,-,AAV
...,...,...,...,...,...,...,...,...,...
479,2019-12-30,3.88,57071.30,2019-09-30 00:00:00,16.36,1.90,2.04,2.10,WHA
480,2020-12-30,3.04,45438.38,2020-09-30 00:00:00,20.01,1.61,1.89,4.40,WHA
481,2021-12-30,3.52,52612.86,2021-09-30 00:00:00,26.19,1.82,1.93,2.85,WHA
482,2022-12-30,4.08,60983.09,2022-09-30 00:00:00,18.83,2.02,2.02,2.46,WHA


In [108]:
table[table.Symbol == "ACE"]

,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
5,2019-12-30,4.22,42942.72,-,62.46,-,-,-,ACE
6,2020-12-30,3.64,37040.64,2020-09-30 00:00:00,24.71,3.15,1.15,-,ACE
7,2021-12-30,3.50,35616.00,2021-09-30 00:00:00,25.58,2.75,1.27,1.37,ACE
8,2022-12-30,2.70,27475.20,2022-09-30 00:00:00,18.75,1.92,1.40,0.75,ACE
9,2023-01-05,2.66,27068.16,2022-09-30 00:00:00,18.41,1.89,1.41,0.76,ACE


In [109]:
save_csv(table,"Statistics")

'succes'

In [110]:
pd.read_csv('Statistics.csv')

,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
0,2019-12-30,2.24,10864.00,2019-09-30 00:00:00,-,0.54,4.12,14.96,AAV
1,2020-12-30,2.36,11446.00,2020-09-30 00:00:00,-,0.76,3.11,-,AAV
2,2021-12-30,2.54,25091.57,2021-09-30 00:00:00,-,3.11,1.71,-,AAV
3,2022-12-30,3.06,37222.71,2022-09-30 00:00:00,-,6.81,0.45,-,AAV
4,2023-01-05,3.08,37466.00,2022-09-30 00:00:00,-,6.86,0.45,-,AAV
...,...,...,...,...,...,...,...,...,...
479,2019-12-30,3.88,57071.30,2019-09-30 00:00:00,16.36,1.90,2.04,2.10,WHA
480,2020-12-30,3.04,45438.38,2020-09-30 00:00:00,20.01,1.61,1.89,4.40,WHA
481,2021-12-30,3.52,52612.86,2021-09-30 00:00:00,26.19,1.82,1.93,2.85,WHA
482,2022-12-30,4.08,60983.09,2022-09-30 00:00:00,18.83,2.02,2.02,2.46,WHA


In [30]:
df = operation_data('DTAC',data_df)

In [31]:
df

Statistics as of,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
0,30 Dec 2019,53.25,126085.94,30 Sep 2019,-,5.17,10.30,1.90,DTAC
1,30 Dec 2020,33.25,78729.72,30 Sep 2020,14.78,3.27,10.16,8.63,DTAC
2,30 Dec 2021,45.75,108327.35,30 Sep 2021,31.26,5.42,8.44,6.54,DTAC
3,30 Dec 2022,46.75,110695.16,30 Sep 2022,46.33,6.19,7.55,4.49,DTAC
4,06 Jan 2023,46.50,110103.21,30 Sep 2022,46.08,6.16,7.55,4.52,DTAC


In [32]:
d = {}
m = [df['Statistics as of'].tolist(),df['Last Price (Baht)'].tolist(),
     df['Market Cap. (M.Baht)'].tolist(),df['F/S Period (As of date)'].tolist()
     ,df['P/E'].tolist(),df['P/BV'].tolist(),df['Book Value per share (Baht)'].tolist()
     ,df['Dividend Yield (%)'].tolist(),df['Symbol'].tolist()]
k = 0
for i in df.columns:
    
    d[i] = m[k]
    k+=1
    
print(d)

{'Statistics as of': ['30 Dec 2019', '30 Dec 2020', '30 Dec 2021', '30 Dec 2022', '06 Jan 2023'], 'Last Price (Baht)': ['53.25', '33.25', '45.75', '46.75', '46.50'], 'Market Cap. (M.Baht)': ['126085.94', '78729.72', '108327.35', '110695.16', '110103.21'], 'F/S Period (As of date)': ['30 Sep 2019', '30 Sep 2020', '30 Sep 2021', '30 Sep 2022', '30 Sep 2022'], 'P/E': ['-', '14.78', '31.26', '46.33', '46.08'], 'P/BV': ['5.17', '3.27', '5.42', '6.19', '6.16'], 'Book Value per share (Baht)': ['10.30', '10.16', '8.44', '7.55', '7.55'], 'Dividend Yield (%)': ['1.90', '8.63', '6.54', '4.49', '4.52'], 'Symbol': ['DTAC', 'DTAC', 'DTAC', 'DTAC', 'DTAC']}


In [33]:
d_c = pd.DataFrame(d)

In [34]:
d_c 

,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
0,30 Dec 2019,53.25,126085.94,30 Sep 2019,-,5.17,10.30,1.90,DTAC
1,30 Dec 2020,33.25,78729.72,30 Sep 2020,14.78,3.27,10.16,8.63,DTAC
2,30 Dec 2021,45.75,108327.35,30 Sep 2021,31.26,5.42,8.44,6.54,DTAC
3,30 Dec 2022,46.75,110695.16,30 Sep 2022,46.33,6.19,7.55,4.49,DTAC
4,06 Jan 2023,46.50,110103.21,30 Sep 2022,46.08,6.16,7.55,4.52,DTAC


# FunctionTH

In [6]:
def get_set50_100_df(driver):
    
    data = driver.page_source
    data_df = pd.read_html(data)[2]
    clean_column = []
    for column in data_df.columns:

        clean_column.append(column.replace("  (Click to sort Ascending)",""))
        
    data_df.columns = clean_column
    data_df = data_df.set_index('Symbol')
    return data_df

def operation_data(stock,dataframe):
    
    Statistics_df = (dataframe.set_index("ค่าสถิติสำคัญ ณ วันที่")).T

    Statistics_df['Symbol'] = stock
    Statistics_df = Statistics_df.reset_index()
    Statistics_df = Statistics_df.rename(columns = {"index":"ค่าสถิติสำคัญ ณ วันที่"})

    return Statistics_df

def get_Statistics_data(driver,stock,all_stock):
    
    stock = stock.split()[0]
    url_target_financial = "https://www.set.or.th/th/market/product/stock/quote/" + stock + "/financial-statement/company-highlights"
    driver.get(url_target_financial)
    driver.execute_script("scrollBy(0,+5000);")
    time.sleep(3)
    
    Statistics_df = driver.page_source
    Statistics_df = pd.read_html(Statistics_df)[1]
    
    Statistics_df = operation_data(stock,Statistics_df)

    all_stock.append(Statistics_df)
    print(stock)
    return driver , all_stock

In [4]:
driver = Set_up_WebScraping("https://www.set.or.th/en/market/index/set100/overview")

C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_17800\2693298640.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\week2\chromedriver.exe")


In [5]:
data_df = get_set50_100_df(driver)

In [8]:
all_Financial = []

for stock in data_df.index :

    
    driver, all_Financial = get_Statistics_data(driver,stock,all_Financial)
    

table_pice = pd.concat(all_Financial)

AAV
ACE
ADVANC
AMATA
AOT
AP
AWC
BAM
BANPU
BBL
BCH
BCP
BCPG
BDMS
BEC
BEM
BGRIM
BH
BLA
BTS
BYD
CBG
CENTEL
CHG
CK
CKP
COM7
CPALL
CPF
CPN
CRC
DELTA
DOHOME
DTAC
EA
EGCO
EPG
ESSO
FORTH
GLOBAL
GPSC
GULF
GUNKUL
HANA
HMPRO
INTUCH
IRPC
IVL
JAS
JMART
JMT
KBANK
KCE
KEX
KKP
KTB
KTC
LH
MEGA
MINT
MTC
NEX
ONEE
OR
ORI
OSP
PLANB
PSL
PTG
PTT
PTTEP
PTTGC
QH
RATCH
RBF
RCL
SABUY
SAWAD
SCB
SCC
SCGP
SINGER
SPALI
SPRC
STA
STARK
STGT
TCAP
THANI
THG
TIDLOR
TIPH
TISCO
TOP
TQM
TRUE
TTB
TU
VGI
WHA


In [9]:
table_pice

ค่าสถิติสำคัญ ณ วันที่,ค่าสถิติสำคัญ ณ วันที่,ราคาล่าสุด (บาท),มูลค่าหลักทรัพย์ตามราคาตลาด (ล้านบาท),วันที่ของงบการเงินที่ใช้คำนวณค่าสถิติ,P/E (เท่า),P/BV (เท่า),มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท),อัตราส่วนเงินปันผลตอบแทน (%),Symbol
0,30 ธ.ค. 2562,2.24,10864.00,30 ก.ย. 2562,-,0.54,4.12,14.96,AAV
1,30 ธ.ค. 2563,2.36,11446.00,30 ก.ย. 2563,-,0.76,3.11,-,AAV
2,30 ธ.ค. 2564,2.54,25091.57,30 ก.ย. 2564,-,3.11,1.71,-,AAV
3,30 ธ.ค. 2565,3.06,37222.71,30 ก.ย. 2565,-,6.81,0.45,-,AAV
4,06 ม.ค. 2566,3.08,37466.00,30 ก.ย. 2565,-,6.86,0.45,-,AAV
...,...,...,...,...,...,...,...,...,...
0,30 ธ.ค. 2562,3.88,57071.30,30 ก.ย. 2562,16.36,1.90,2.04,2.10,WHA
1,30 ธ.ค. 2563,3.04,45438.38,30 ก.ย. 2563,20.01,1.61,1.89,4.40,WHA
2,30 ธ.ค. 2564,3.52,52612.86,30 ก.ย. 2564,26.19,1.82,1.93,2.85,WHA
3,30 ธ.ค. 2565,4.08,60983.09,30 ก.ย. 2565,18.83,2.02,2.02,2.46,WHA


In [10]:
save_csv(table_pice,"Statistics_th")

'succes'

In [22]:
all_Statistics_year = []
all_Statistics_month = []
i = 0
for stock in stock_name:

        print(i,' ',stock)
        all_Statistics_year,all_Statistics_month = self.get_Statistics_data(stock,all_Statistics_year,all_Statistics_month)
        i+=1
        

NameError: name 'stock_name' is not defined

# UPDATE 

In [4]:
import sqlite3

In [5]:
location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\Set_dataframe_to_sqlite3\share.sqlite'

In [6]:
con = sqlite3.connect(location_db)
cur = con.cursor()
# con.close()

In [7]:
stock = ['ACE', 'ADVANC', 'AEONTS', 'AMATA', 'AOT', 'AP', 'AWC', 'BAM',
       'BANPU', 'BBL', 'BCH', 'BCP', 'BCPG', 'BDMS', 'BEC', 'BEM',
       'BGRIM', 'BH', 'BLA', 'BTS', 'CBG', 'CENTEL', 'CHG', 'CK', 'CKP',
       'COM7', 'CPALL', 'CPF', 'CPN', 'CRC', 'DOHOME', 'DTAC', 'EA',
       'EGCO', 'EPG', 'ESSO', 'FORTH', 'GLOBAL', 'GPSC', 'GULF', 'GUNKUL',
       'HANA', 'HMPRO', 'INTUCH', 'IRPC', 'IVL', 'JMART', 'JMT', 'KBANK',
       'KCE', 'KEX', 'KKP', 'KTB', 'KTC', 'LH', 'MAJOR', 'MEGA', 'MINT',
       'MTC', 'ONEE', 'OR', 'ORI', 'OSP', 'PLANB', 'PSL', 'PTG', 'PTT',
       'PTTEP', 'PTTGC', 'QH', 'RATCH', 'RBF', 'RCL', 'SAWAD', 'SCB',
       'SCC', 'SCGP', 'SINGER', 'SPALI', 'SPRC', 'STA', 'STARK', 'STEC',
       'STGT', 'SUPER', 'SYNEX', 'TASCO', 'TCAP', 'THANI', 'TIDLOR',
       'TIPH', 'TISCO', 'TOP', 'TQM', 'TRUE', 'TTA', 'TTB', 'TU', 'VGI',
       'WHA']

In [8]:
def operation_data_Statistics(stock,dataframe,symbol_id):

    Statistics_df = (dataframe.set_index("Statistics as of")).T
    
    Statistics_df['SymbolId'] = symbol_id[stock]
    Statistics_df = Statistics_df.reset_index()
    Statistics_df = Statistics_df.rename(columns = {"index":"Statistics as of"})

    return Statistics_df

def get_Statistics_data(driver,stock,sh_id):

    stock = stock.split()[0]
    url_target_financial = "https://www.set.or.th/en/market/product/stock/quote/" + stock + "/financial-statement/company-highlights"
    driver.get(url_target_financial)
        
    driver.execute_script("scrollBy(0,+5000);")
    time.sleep(3)
    
    Statistics_df = driver.page_source
    Statistics_df = pd.read_html(Statistics_df)[1]

    Statistics_df = operation_data_Statistics(stock,Statistics_df,sh_id)
       
    Statistics_df['Statistics as of'] = pd.to_datetime(Statistics_df['Statistics as of'])
    Statistics_df['F/S Period (As of date)'] = pd.to_datetime(Statistics_df['F/S Period (As of date)'])
        
    return driver,Statistics_df

In [11]:
print(' ','MAX')
driver,Statistics_df = get_Statistics_data(driver,'MAX',sh_id)     

  MAX


In [12]:
Statistics_df

Statistics as of,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),SymbolId
0,2019-12-30,0.02,597.01,2019-09-30,-,0.43,0.05,-,4
1,2020-12-30,0.01,856.69,2020-09-30,-,0.46,0.02,-,4
2,2021-12-30,0.01,-,2020-09-30,-,-,0.02,-,4
3,2022-12-30,-,-,2021-03-31,-,-,0.02,-,4
4,2023-01-24,-,-,2021-12-31,-,-,0.02,-,4


In [125]:
d = int(Statistics_df.iloc[[0]]['SymbolId'])

In [126]:
d

374

In [50]:
d = Statistics_df['Statistics as of'].tolist()

In [162]:
sql = """select "{}" From {} where SymbolId = '{}' and "{}" = '{}' LIMIT 1
""".format("Statistics as of",'Satistics_year',int(Statistics_df.iloc[[0]]['SymbolId']),
           "Statistics as of",'2019-12-30' + ' 00:00:00')

In [163]:
cur.execute(sql)
records = cur.fetchall()

In [164]:
records

[('2019-12-30 00:00:00',)]

In [13]:
def Check_Statistics(table,df,time):
    
    location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\Set_dataframe_to_sqlite3\share.sqlite'
    con = sqlite3.connect(location_db)
    cur = con.cursor()
    
    sql = """select "{}" From {} where SymbolId = '{}' and "{}" = '{}' LIMIT 1
    """.format("Statistics as of",table,int(df.iloc[[0]]['SymbolId']),
               "Statistics as of",str(time))

    cur.execute(sql)
    records = cur.fetchall()
    con.close()

    return not(bool(records))

In [14]:
# Satistics_year'

def new_Statistics(Statistics_df):
    data_new_year = []
    r = 0
    for time in (Statistics_df['Statistics as of'].tolist())[:-1]:

        Nhave = Check_Statistics('Satistics_year',Statistics_df.iloc[[r]],time)

        if Nhave :

            data_new_year.append(Statistics_df.iloc[[r]])

        r+=1
    
    if data_new_year :
        
        return pd.concat(data_new_year)
    
    return pd.DataFrame({'Statistics as of': []
        , 'Last Price (Baht)': []
        , 'Market Cap. (M.Baht)': []
        , 'F/S Period (As of date)': []
        , 'P/E': [], 'P/BV': []
        , 'Book Value per share (Baht)': [], 'Dividend Yield (%)': []
        , 'SymbolId': []})

In [15]:
new_Statistics(Statistics_df)

Statistics as of,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),SymbolId
0,2019-12-30,0.02,597.01,2019-09-30,-,0.43,0.05,-,4
1,2020-12-30,0.01,856.69,2020-09-30,-,0.46,0.02,-,4
2,2021-12-30,0.01,-,2020-09-30,-,-,0.02,-,4
3,2022-12-30,-,-,2021-03-31,-,-,0.02,-,4


In [ ]:
# Satistics_day
time = str((Statistics_df['Statistics as of'].tolist())[-1])
e = Check_Statistics('Satistics_day',Statistics_df.iloc[[-1]],time)

In [54]:
new_Statistics(Statistics_df)

StopIteration: 

In [52]:
data = {'Statistics as of': ['2019-12-30', '2019-12-30', '2019-12-30', '2019-12-30', '2023-01-06']
        , 'Last Price (Baht)': ['53.25', '33.25', '45.75', '46.75', '46.50']
        , 'Market Cap. (M.Baht)': ['126085.94', '78729.72', '108327.35', '110695.16', '110103.21']
        , 'F/S Period (As of date)': ['2019-09-30', '2020-09-30', '2020-09-30', '2021-03-31', '30 Sep 2022']
        , 'P/E': ['-', '14.78', '31.26', '46.33', '46.08'], 'P/BV': ['5.17', '3.27', '5.42', '6.19', '6.16']
        , 'Book Value per share (Baht)': ['10.30', '10.16', '8.44', '7.55', '7.55'], 'Dividend Yield (%)': ['1.90', '8.63', '6.54', '4.49', '4.52']
        , 'Symbol': ['4', '4', '4', '4', '4']}
        
pd.DataFrame(data)

,Statistics as of,Last Price (Baht),Market Cap. (M.Baht),F/S Period (As of date),P/E,P/BV,Book Value per share (Baht),Dividend Yield (%),Symbol
0,2019-12-30,53.25,126085.94,2019-09-30,-,5.17,10.30,1.90,4
1,2019-12-30,33.25,78729.72,2020-09-30,14.78,3.27,10.16,8.63,4
2,2019-12-30,45.75,108327.35,2020-09-30,31.26,5.42,8.44,6.54,4
3,2019-12-30,46.75,110695.16,2021-03-31,46.33,6.19,7.55,4.49,4
4,2023-01-06,46.50,110103.21,30 Sep 2022,46.08,6.16,7.55,4.52,4


In [10]:
sh_id = {'EE': 1,
 'GFPT': 2,
 'LEE': 3,
 'MAX': 4,
 'NER': 5,
 'PPPM': 6,
 'STA': 7,
 'TEGH': 8,
 'TFM': 9,
 'TRUBB': 10,
 'TWPC': 11,
 'UPOIC': 12,
 'UVAN': 13,
 'VPO': 14,
 'AAI': 15,
 'APURE': 16,
 'ASIAN': 17,
 'BR': 18,
 'BRR': 19,
 'BTG': 20,
 'CBG': 21,
 'CFRESH': 22,
 'CH': 23,
 'CHOTI': 24,
 'CM': 25,
 'CPF': 26,
 'CPI': 27,
 'F&D': 28,
 'GLOCON': 29,
 'HTC': 30,
 'ICHI': 31,
 'ITC': 32,
 'JDF': 33,
 'KBS': 34,
 'KSL': 35,
 'KTIS': 36,
 'LST': 37,
 'M': 38,
 'MALEE': 39,
 'MINT': 40,
 'NRF': 41,
 'NSL': 42,
 'OISHI': 43,
 'OSP': 44,
 'PB': 45,
 'PLUS': 46,
 'PM': 47,
 'PRG': 48,
 'RBF': 49,
 'SAPPE': 50,
 'SAUCE': 51,
 'SFP': 52,
 'SNNP': 53,
 'SNP': 54,
 'SORKON': 55,
 'SSC': 56,
 'SSF': 57,
 'SST': 58,
 'SUN': 59,
 'TC': 60,
 'TFG': 61,
 'TFMAMA': 62,
 'TIPCO': 63,
 'TKN': 64,
 'TU': 65,
 'TVO': 66,
 'W': 67,
 'ZEN': 68,
 'AFC': 69,
 'AURA': 70,
 'BTNC': 71,
 'CPH': 72,
 'CPL': 73,
 'NC': 74,
 'PAF': 75,
 'PDJ': 76,
 'PG': 77,
 'SABINA': 78,
 'SAWANG': 79,
 'SUC': 80,
 'TNL': 81,
 'TR': 82,
 'TTI': 83,
 'TTT': 84,
 'UPF': 85,
 'WACOAL': 86,
 'WFX': 87,
 'AJA': 88,
 'DTCI': 89,
 'FANCY': 90,
 'FTI': 91,
 'KYE': 92,
 'L&E': 93,
 'MODERN': 94,
 'OGC': 95,
 'ROCK': 96,
 'SIAM': 97,
 'TCMC': 98,
 'TSR': 99,
 'APCO': 100,
 'BIZ': 101,
 'DDD': 102,
 'JCT': 103,
 'KISS': 104,
 'NV': 105,
 'OCC': 106,
 'S&J': 107,
 'STGT': 108,
 'STHAI': 109,
 'TNR': 110,
 'TOG': 111,
 '3K-BAT': 112,
 'ACG': 113,
 'AH': 114,
 'CWT': 115,
 'EASON': 116,
 'GYT': 117,
 'HFT': 118,
 'IHL': 119,
 'INGRS': 120,
 'IRC': 121,
 'PCSGH': 122,
 'POLY': 123,
 'SAT': 124,
 'SPG': 125,
 'STANLY': 126,
 'TKT': 127,
 'TNPC': 128,
 'TRU': 129,
 'TSC': 130,
 'ALLA': 131,
 'ASEFA': 132,
 'CPT': 133,
 'CRANE': 134,
 'CTW': 135,
 'FMT': 136,
 'HTECH': 137,
 'KKC': 138,
 'PK': 139,
 'SNC': 140,
 'STARK': 141,
 'TCJ': 142,
 'TPCS': 143,
 'VARO': 144,
 'UTP': 145,
 'BCT': 146,
 'CMAN': 147,
 'GC': 148,
 'GGC': 149,
 'GIFT': 150,
 'IVL': 151,
 'NFC': 152,
 'PATO': 153,
 'PMTA': 154,
 'PTTGC': 155,
 'SUTHA': 156,
 'TCCC': 157,
 'TPA': 158,
 'UAC': 159,
 'UP': 160,
 'AJ': 161,
 'ALUCON': 162,
 'BGC': 163,
 'CSC': 164,
 'NEP': 165,
 'PTL': 166,
 'SCGP': 167,
 'SFLEX': 168,
 'SITHAI': 169,
 'SLP': 170,
 'SMPC': 171,
 'SPACK': 172,
 'TCOAT': 173,
 'TFI': 174,
 'THIP': 175,
 'TMD': 176,
 'TOPP': 177,
 'TPAC': 178,
 'TPBI': 179,
 'TPP': 180,
 '2S': 181,
 'AMC': 182,
 'BSBM': 183,
 'CEN': 184,
 'CITY': 185,
 'CSP': 186,
 'GJS': 187,
 'GSTEEL': 188,
 'INOX': 189,
 'LHK': 190,
 'MCS': 191,
 'MILL': 192,
 'PAP': 193,
 'PERM': 194,
 'SAM': 195,
 'SMIT': 196,
 'SSSC': 197,
 'TGPRO': 198,
 'THE': 199,
 'TMT': 200,
 'TSTH': 201,
 'TWP': 202,
 'TYCN': 203,
 'CCP': 204,
 'COTTO': 205,
 'DCC': 206,
 'DCON': 207,
 'DRT': 208,
 'EPG': 209,
 'GEL': 210,
 'PPP': 211,
 'Q-CON': 212,
 'SCC': 213,
 'SCCC': 214,
 'SCP': 215,
 'SKN': 216,
 'STECH': 217,
 'TASCO': 218,
 'TOA': 219,
 'TPIPL': 220,
 'UMI': 221,
 'VNG': 222,
 'WIIK': 223,
 'A': 224,
 'AMATA': 225,
 'AMATAV': 226,
 'ANAN': 227,
 'AP': 228,
 'APEX': 229,
 'AQ': 230,
 'ASW': 231,
 'AWC': 232,
 'BLAND': 233,
 'BRI': 234,
 'BROCK': 235,
 'CGD': 236,
 'CI': 237,
 'CMC': 238,
 'CPN': 239,
 'ESTAR': 240,
 'EVER': 241,
 'FPT': 242,
 'GLAND': 243,
 'J': 244,
 'JCK': 245,
 'KC': 246,
 'LALIN': 247,
 'LH': 248,
 'LPN': 249,
 'MBK': 250,
 'MJD': 251,
 'MK': 252,
 'NCH': 253,
 'NNCL': 254,
 'NOBLE': 255,
 'NUSA': 256,
 'NVD': 257,
 'ORI': 258,
 'PACE': 259,
 'PEACE': 260,
 'PF': 261,
 'PIN': 262,
 'PLAT': 263,
 'POLAR': 264,
 'PRECHA': 265,
 'PRIN': 266,
 'PSH': 267,
 'QH': 268,
 'RABBIT': 269,
 'RICHY': 270,
 'RML': 271,
 'ROJNA': 272,
 'S': 273,
 'SA': 274,
 'SAMCO': 275,
 'SC': 276,
 'SENA': 277,
 'SIRI': 278,
 'SPALI': 279,
 'UV': 280,
 'WHA': 281,
 'WIN': 282,
 'AIMCG': 283,
 'AIMIRT': 284,
 'ALLY': 285,
 'AMATAR': 286,
 'B-WORK': 287,
 'BAREIT': 288,
 'BKKCP': 289,
 'BOFFICE': 290,
 'CPNCG': 291,
 'CPNREIT': 292,
 'CPTGF': 293,
 'CTARAF': 294,
 'DREIT': 295,
 'ERWPF': 296,
 'FTREIT': 297,
 'FUTUREPF': 298,
 'GAHREIT': 299,
 'GROREIT': 300,
 'GVREIT': 301,
 'HPF': 302,
 'HYDROGEN': 303,
 'IMPACT': 304,
 'INETREIT': 305,
 'KPNPF': 306,
 'KTBSTMR': 307,
 'LHHOTEL': 308,
 'LHPF': 309,
 'LHSC': 310,
 'LPF': 311,
 'LUXF': 312,
 'M-II': 313,
 'M-PAT': 314,
 'M-STOR': 315,
 'MIPF': 316,
 'MIT': 317,
 'MJLF': 318,
 'MNIT': 319,
 'MNIT2': 320,
 'MNRF': 321,
 'POPF': 322,
 'PPF': 323,
 'PROSPECT': 324,
 'QHHR': 325,
 'QHOP': 326,
 'QHPF': 327,
 'SHREIT': 328,
 'SIRIP': 329,
 'SPRIME': 330,
 'SRIPANWA': 331,
 'SSPF': 332,
 'SSTRT': 333,
 'TIF1': 334,
 'TLHPF': 335,
 'TNPF': 336,
 'TPRIME': 337,
 'TTLPF': 338,
 'TU-PF': 339,
 'URBNPF': 340,
 'WHABT': 341,
 'WHAIR': 342,
 'WHART': 343,
 'APCS': 344,
 'BJCHI': 345,
 'BKD': 346,
 'CIVIL': 347,
 'CK': 348,
 'CNT': 349,
 'EMC': 350,
 'ITD': 351,
 'NWR': 352,
 'PLE': 353,
 'PREB': 354,
 'PYLON': 355,
 'RT': 356,
 'SEAFCO': 357,
 'SQ': 358,
 'SRICHA': 359,
 'STEC': 360,
 'STI': 361,
 'STPI': 362,
 'SYNTEC': 363,
 'TEAMG': 364,
 'TEKA': 365,
 'TPOLY': 366,
 'TRC': 367,
 'TRITN': 368,
 'TTCL': 369,
 'UNIQ': 370,
 'WGE': 371,
 '7UP': 372,
 'ACC': 373,
 'ACE': 374,
 'AGE': 375,
 'AI': 376,
 'AIE': 377,
 'AKR': 378,
 'BAFS': 379,
 'BANPU': 380,
 'BBGI': 381,
 'BCP': 382,
 'BCPG': 383,
 'BGRIM': 384,
 'BPP': 385,
 'BRRGIF': 386,
 'CKP': 387,
 'CV': 388,
 'DEMCO': 389,
 'EA': 390,
 'EASTW': 391,
 'EGATIF': 392,
 'EGCO': 393,
 'EP': 394,
 'ESSO': 395,
 'ETC': 396,
 'GPSC': 397,
 'GREEN': 398,
 'GULF': 399,
 'GUNKUL': 400,
 'IFEC': 401,
 'IRPC': 402,
 'JR': 403,
 'KBSPIF': 404,
 'LANNA': 405,
 'MDX': 406,
 'NOVA': 407,
 'OR': 408,
 'PCC': 409,
 'PRIME': 410,
 'PTG': 411,
 'PTT': 412,
 'PTTEP': 413,
 'QTC': 414,
 'RATCH': 415,
 'RPC': 416,
 'SCG': 417,
 'SCI': 418,
 'SCN': 419,
 'SGP': 420,
 'SKE': 421,
 'SOLAR': 422,
 'SPCG': 423,
 'SPRC': 424,
 'SSP': 425,
 'SUPER': 426,
 'SUPEREIF': 427,
 'SUSCO': 428,
 'TAE': 429,
 'TCC': 430,
 'TGE': 431,
 'TOP': 432,
 'TPIPP': 433,
 'TSE': 434,
 'TTW': 435,
 'UBE': 436,
 'WHAUP': 437,
 'WP': 438,
 'THL': 439,
 'B52': 440,
 'BEAUTY': 441,
 'BIG': 442,
 'BJC': 443,
 'COM7': 444,
 'CPALL': 445,
 'CPW': 446,
 'CRC': 447,
 'CSS': 448,
 'DOHOME': 449,
 'FN': 450,
 'FTE': 451,
 'GLOBAL': 452,
 'HMPRO': 453,
 'ICC': 454,
 'ILM': 455,
 'IT': 456,
 'KAMART': 457,
 'LOXLEY': 458,
 'MAKRO': 459,
 'MC': 460,
 'MEGA': 461,
 'MIDA': 462,
 'MOSHI': 463,
 'RS': 464,
 'RSP': 465,
 'SABUY': 466,
 'SCM': 467,
 'SINGER': 468,
 'SPC': 469,
 'SPI': 470,
 'SVT': 471,
 'AHC': 472,
 'BCH': 473,
 'BDMS': 474,
 'BH': 475,
 'CHG': 476,
 'CMR': 477,
 'EKH': 478,
 'KDH': 479,
 'LPH': 480,
 'M-CHAI': 481,
 'NEW': 482,
 'NTV': 483,
 'PR9': 484,
 'PRINC': 485,
 'RAM': 486,
 'RJH': 487,
 'RPH': 488,
 'SKR': 489,
 'THG': 490,
 'VIBHA': 491,
 'VIH': 492,
 'WPH': 493,
 'AMARIN': 494,
 'AQUA': 495,
 'AS': 496,
 'BEC': 497,
 'FE': 498,
 'GPI': 499,
 'GRAMMY': 500,
 'JKN': 501,
 'MACO': 502,
 'MAJOR': 503,
 'MATCH': 504,
 'MATI': 505,
 'MCOT': 506,
 'MONO': 507,
 'MPIC': 508,
 'NATION': 509,
 'ONEE': 510,
 'PLANB': 511,
 'POST': 512,
 'PRAKIT': 513,
 'PTECH': 514,
 'SE-ED': 515,
 'TKS': 516,
 'VGI': 517,
 'WAVE': 518,
 'WORK': 519,
 'BWG': 520,
 'GENCO': 521,
 'PRO': 522,
 'SISB': 523,
 'SO': 524,
 'ASIA': 525,
 'BEYOND': 526,
 'CENTEL': 527,
 'CSR': 528,
 'DUSIT': 529,
 'ERW': 530,
 'GRAND': 531,
 'LRH': 532,
 'MANRIN': 533,
 'OHTL': 534,
 'ROH': 535,
 'SHANG': 536,
 'SHR': 537,
 'VRANDA': 538,
 'AAV': 539,
 'AOT': 540,
 'ASIMAR': 541,
 'B': 542,
 'BA': 543,
 'BEM': 544,
 'BIOTEC': 545,
 'BTS': 546,
 'BTSGIF': 547,
 'DMT': 548,
 'III': 549,
 'JWD': 550,
 'KEX': 551,
 'KIAT': 552,
 'KWC': 553,
 'MENA': 554,
 'NOK': 555,
 'NYT': 556,
 'PORT': 557,
 'PRM': 558,
 'PSL': 559,
 'RCL': 560,
 'TFFIF': 561,
 'THAI': 562,
 'TSTE': 563,
 'TTA': 564,
 'WICE': 565,
 'CCET': 566,
 'DELTA': 567,
 'HANA': 568,
 'KCE': 569,
 'METCO': 570,
 'NEX': 571,
 'SMT': 572,
 'SVI': 573,
 'TEAM': 574,
 'ADVANC': 575,
 'AIT': 576,
 'ALT': 577,
 'AMR': 578,
 'BLISS': 579,
 'DIF': 580,
 'DTAC': 581,
 'DTCENT': 582,
 'FORTH': 583,
 'HUMAN': 584,
 'ILINK': 585,
 'INET': 586,
 'INSET': 587,
 'INTUCH': 588,
 'ITEL': 589,
 'JAS': 590,
 'JASIF': 591,
 'JMART': 592,
 'JTS': 593,
 'MFEC': 594,
 'MSC': 595,
 'PT': 596,
 'SAMART': 597,
 'SAMTEL': 598,
 'SDC': 599,
 'SIS': 600,
 'SKY': 601,
 'SVOA': 602,
 'SYMC': 603,
 'SYNEX': 604,
 'THCOM': 605,
 'TKC': 606,
 'TRUE': 607,
 'TWZ': 608,
 'AEONTS': 609,
 'BAM': 610,
 'BBL': 611,
 'BLA': 612,
 'JMT': 613,
 'KBANK': 614,
 'KKP': 615,
 'KTB': 616,
 'KTC': 617,
 'MTC': 618,
 'SAWAD': 619,
 'SCB': 620,
 'TCAP': 621,
 'THANI': 622,
 'TIDLOR': 623,
 'TIPH': 624,
 'TISCO': 625,
 'TQM': 626,
 'TTB': 627}